# Projeto 01 - Car Price Prediction

### Objetivo
O objetivo é estimar um valor de venda para novos veículos.

### Base de dados
O conjunto de dados foi retirado do site CarDekho e a base de treino contém 5.689 carros. Já a base de teste possui 2.375.




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
random.seed(202101)

In [ ]:
!pip3 install catboost

## Análise dos dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Aprendizado de Máquina/Bases/train_car_details.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Aprendizado de Máquina/Bases/test_car_details.csv")

Os dados são compostos pelas variáveis:


* Variaveis quantitativas:<br>
 * Ano de fabricacao do carro (year)
 * Qtd de Km dirigidos (km_driven)
 * Potência máxima do motor (max_power)
 * Qtd de acentos (seats)
 * Quilometragem por litro (mileage)
 * Potencia do motor (engine)
 * Torque: responsável pela capacidade do motor produzir força motriz, ou seja, o movimento giratório
 * Preço de venda (selling_price) **Valor a ser predito**

* Variaveis qualitativas:
 * nome do carro (name)
 * tipo de combustivel utilizado (fuel)
 * tipo de vendendor (seller_type)
 * transmission: câmbio automático ou manual

 * Quantos donos ja possuiram o carro (owner)

### Treino

In [ ]:
df_train.head()

,Id,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,1,Hyundai Santro GLS I - Euro I,1999,80000,110000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN
1,2,Maruti Ertiga VDI,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77 kmpl,1248 CC,88.76 bhp,200Nm@ 1750rpm,7.0
2,3,BMW 3 Series 320d Luxury Line,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62 kmpl,1995 CC,187.74 bhp,400Nm@ 1750-2500rpm,5.0
3,4,Tata New Safari DICOR 2.2 EX 4x2,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57 kmpl,2179 CC,138.1 bhp,320Nm@ 1700-2700rpm,7.0
4,5,Toyota Fortuner 3.0 Diesel,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5 kmpl,2982 CC,171 bhp,343Nm@ 1400-3400rpm,7.0


In [ ]:
#Porcentagem de nan por atributo
print(round(100*df_train.isna().sum()/len(df_train), 2))

Id               0.00
name             0.00
year             0.00
selling_price    0.00
km_driven        0.00
fuel             0.00
seller_type      0.00
transmission     0.00
owner            0.00
mileage          2.76
engine           2.76
max_power        2.65
torque           2.78
seats            2.76
dtype: float64


In [ ]:
pd.set_option('display.max_rows', 200)
df_train[df_train['mileage'].isnull()]
#pd.reset_option('display.max_rows')

Pelo fatos dos NaN's estar majoritariamente presente nas mesmas linhas e por representar um baixo volume em relação ao total (menos de 3%), tais linhas serão retiradas.

In [ ]:
print(f'Quantidade de linhas totais: ', df_train.shape[0])
# Remove as linhas com NaN
df_train = df_train.dropna(axis=0)   
print(f'Quantidade de linhas após retirada dos NaNs: ', df_train.shape[0])
#Aproximadamente 3% de linhas eliminadas

Quantidade de linhas totais:  5689
Quantidade de linhas após retirada dos NaNs:  5531


### Teste

In [ ]:
df_test.head()

,Id,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,1,Tata Indigo LS,2008,60000,Diesel,Individual,Manual,Third Owner,17.0 kmpl,1405 CC,70 bhp,"13.5@ 2,500(kgm@ rpm)",5.0
1,2,Maruti Baleno Delta 1.2,2019,30000,Petrol,Individual,Manual,First Owner,21.4 kmpl,1197 CC,83.1 bhp,115Nm@ 4000rpm,5.0
2,3,Maruti Swift Dzire AMT ZDI,2017,80000,Diesel,Individual,Automatic,First Owner,28.4 kmpl,1248 CC,74.02 bhp,190Nm@ 2000rpm,5.0
3,4,Mahindra XUV500 W10 2WD,2018,30000,Diesel,Individual,Manual,First Owner,16.0 kmpl,2179 CC,140 bhp,330Nm@ 1600-2800rpm,7.0
4,5,Hyundai Elite i20 Asta Option BSIV,2019,35000,Petrol,Individual,Manual,First Owner,18.6 kmpl,1197 CC,81.86 bhp,117nm@ 4000rpm,5.0


Tanto a base de treino quanto a base de teste possuem as mesmas colunas.

In [ ]:
#Porcentagem de nan por atributo
print(round(100*df_test.isna().sum()/len(df_train), 2))

Para facilitar a correção de algumas variáveis tanto do treino quanto do teste, serão criadas funções.

In [ ]:
df_train = df_train.iloc[:,1:] #retirando a coluna de id do treino
df_test = df_test.iloc[:,1:] #retirando a coluna Id do teste

## Pré-Processamento dos dados